In [40]:
import re
import pandas as pd

def extract_params_from_file_name(df: pd.DataFrame):
    df['maxNewTokensFactor'] = df['file'].apply(lambda x: re.search(r'maxNewTokensFactor(\d+)', x).group(1))
    df['nShotsInference'] = df['file'].apply(lambda x: re.search(r'nShotsInference(\d+)', x).group(1))
    #df['layer'] = df['file'].apply(lambda x: re.search(r'adapters_(\s+)', x).group(1))
    df['model'] = df['file'].apply(lambda x: str(x.split('_adapters_')[0].split('nShotsInference')[1][2:]))
    df['training_params_string'] = df['file'].apply(lambda x: x.split('adapters_')[1])
    df['nbit'] = df['training_params_string'].apply(lambda x: int(x.split('_')[1]))
    df['bnb_4bit_compute_dtype'] = df['training_params_string'].apply(lambda x: x.split('_')[2])
    df['r'] = df['training_params_string'].apply(lambda x: int(x.split('_')[3]))
    df['lora_alpha'] = df['training_params_string'].apply(lambda x: int(x.split('_')[4]))
    df['lora_dropout'] = df['training_params_string'].apply(lambda x: float(x.split('_')[5]))
    df['gradient_accumulation_steps'] = df['training_params_string'].apply(lambda x: int(x.split('_')[6]))
    df['learning_rate'] = df['training_params_string'].apply(lambda x: float(x.split('_')[7].split('.')[0]))
    df = df.drop(columns=['training_params_string', 'file'])
    return df


In [46]:
mistral_8bit = pd.read_csv('data/evaluation_results/mistral_8bit.csv')
mistral_8bit = extract_params_from_file_name(mistral_8bit)
mistral_4bit = pd.read_csv('data/evaluation_results/mistral_4bit.csv')
mistral_4bit = extract_params_from_file_name(mistral_4bit)
llama7b_8bit = pd.read_csv('data/evaluation_results/llama7b_8bit.csv')
llama7b_8bit = extract_params_from_file_name(llama7b_8bit)

pd.concat([mistral_4bit, mistral_8bit, llama7b_8bit]).to_csv('data/evaluation_results/joint_results.csv')